In [126]:
import re

#first read NC_045512.fasta txt into list form to perform mutated operations

input1 = open("/Users/lindsayreisman/Documents/NC_045512.fasta", 'r')
header = input1.readline() #skip first line
NC_045512_list_version = [b for line in input1.read().split('\n') for b in line]
input1.close()


#mutation stage
#read SNP_coordinates txt to find coordinates to change

input2 = open("/Users/lindsayreisman/Documents/countResult.txt", "r")
input2.readline() #skip first line
count_dict = {}

for line in input2.readlines():
    
    coordinate, change = line.split(":")
    count_dict[int(coordinate)] = [change[1],change[4]] #[num1,num2]
    
    #Note int(coordinate) -1 to account for zero indexing
    NC_045512_list_version[int(coordinate) - 1] = change[4]
    
#print(count_dict)    
input2.close()

NC_045512_mutated_string_final = "".join(NC_045512_list_version)

# write mutated version to file
with open("/Users/lindsayreisman/Documents/mutated_NC_045512.fasta.txt", "w") as f:
    
    f.write(header)
    f.write(NC_045512_mutated_string_final)

# new_str = old_str[:i] + "b" + old_str[i+1:]
def map_strain(): #iterate through each strain output file and map to mutated sequence 
    g = ['E','N','orf1ab','S','M','ORF10','ORF3a','ORF8'] #using variable 
    directory = '/Users/lindsayreisman/Documents/Final Project/'
    for gene in g:  
        ## make directory for each gene
        os.mkdir(directory + '/Mutated Gene Strains/' + gene)
        for i in os.listdir(directory+gene):
            #print(i)
            files = open(directory+gene+'/'+i)
            r = files.read()
            defline = r.split('\n')[0]
            seq = ''.join(r.split('\n')[1:])
            range1= int(r.split('-')[0].split(':')[2])
            range2= int(r.split('-')[1].split('|')[0])
            #print(seq)
            #print(range1,range2)
            # print(seq[range1-range1])
            out = open(directory + '/Mutated Gene Strains/' + g + '/mutated_'+i, "w")
            for x in range(range1, range2):
                # print(seq[x-range1])
                
                if x in count_dict:
                    if seq[x-range1-1] == count_dict[x][0]:
                        seq = seq[:x-range1-1] + count_dict[x][1] + seq[x-range1:]
                        #print(x-range1-1,count_dict[x], seq[x-range1-1:])
                    #print(count_dict[x])
            out.write(defline+'\n'+seq)
            #print(seq)
            
    
map_strain()

In [130]:
content = ''
directory = '/Users/lindsayreisman/Documents/Final Project/Mutated Gene Strains/orf1ab/'
for i in os.listdir(directory):
    files = open(directory+i)
    #print(i)
    r = files.read()
    content += r + '\n'+'\n'
    
out = open("/Users/lindsayreisman/Documents/Final Project/ORF1ab_CDS.fasta",'w')
out.write(content)

4528616

In [173]:
from Bio import Entrez
import re
import os
Entrez.email = "ljr390@nyu.edu"
content = ''
with open("/Users/lindsayreisman/Documents/Final Project/ORF1ab_CDS.fasta", "r") as handle:
    genes = handle.read() 
    #j = ''.join(genes) #join from readlines
    all_dem_genes = genes.split('\n\n')
    for dat_gene in all_dem_genes[:-1]:
        n_split = dat_gene.split(':') #split based on colon 
        #print(n_split[:10])
        locus = n_split[1] #capture the second index to query in Entrez 
        #print(locus)
    #print(IDlist)
        handle = Entrez.efetch(db ="nucleotide", id= locus  ,rettype="gb", retmode="xml")
        record = Entrez.read(handle)
        #print(gene,seq,locus)
        #for r in record[0]:
        #    print(r)
        #print(len(record[0]['GBSeq_feature-table']))
        feature_table = record[0]['GBSeq_feature-table'] # find the data to use in the in the directory 
        for f in feature_table:
            if f['GBFeature_key'] == 'CDS':  
                #print(f)
                        
                for G in f['GBFeature_quals']:
                    if G['GBQualifier_name']== 'gene':
                         gene_v1 = G['GBQualifier_value'].upper()
                    if G['GBQualifier_name']== 'translation':
                         gene_v2 = G['GBQualifier_value']
                    if G['GBQualifier_name']== 'protein_id':
                         gene_v3 = G['GBQualifier_value']
                    if G['GBQualifier_name']== 'product':
                        gene_v4 = G['GBQualifier_value']

                content += 'gene:' + gene_v1 + '\t' + 'Protein Id:' + gene_v3 + '\t'+ 'Protein Name:'+ gene_v4 + '\n' + gene_v2 + '\n' + '\n'

            
gene_file = "/Users/lindsayreisman/Documents/Final Project/ORF1ab_PP.fasta" #create an output to write to 

out = open(gene_file, 'w')
out.write(content)
            #print(gene_v1, os.listdir('./Final Project/Translated Data/'))
            #step_1 = ['E','N','orf1ab','S','M','ORF10','ORF3a','ORF8'] 
    
#                 if gene_v1 not in os.listdir('./Final Project/Translated Data/'):
#                     if gene_v1.lower() not in os.listdir('./Final Project/Translated Data/'):
#                         os.mkdir('./Final Project/Translated Data/' + gene_v1)
#                 gene_file = "./Final Project/Translated Data/"+gene_v1+'/'+(gene_v1+'_'+gene_v3)+'.txt'#create an output to write to 
#                 out = open(gene_file, 'w') #the file being written to 
#                 out.write(gene_v1 + '\t' + gene_v3 + '\n'+ gene_v2)

                    


2168450